# data.world datacamp tutorial

> #### Setup  

> Before running data.world notebooks for the first time, you'll need to:  
1. Install data.world's Python package, including optional `pandas` dependencies: 
```shell
pip install datadotworld[pandas]
```
1. Obtain an API access token at https://data.world/settings/advanced
1. Store API access token using the `dw` command-line tool: 
```shell
dw configure
```

> Once your environment is set up, these steps do not need to be repeated for other data.world notebooks.

In [ ]:
# import datadotworld module and pp.pprint

import datadotworld as dw
import os
import pprint as pp

In [ ]:
# import matplotlib for plots

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# load_dataset examples

dataset = dw.load_dataset('https://data.world/stephen-hoover/chicago-city-council-votes')

In [ ]:
# describe examples: describe dataset

dataset.describe()

In [ ]:
# describe examples: describe specific dataset resource

dataset.describe('alderman_votes')

In [ ]:
# Accessing the data
## After loading a dataset object, you can access the data via: `raw_data`, `tables`, or `dataframes`.
## Each of these returns a dictionary of values: `bytes`, `list` and `pandas.DataFrame` objects, respectively.

votes_dataframe = dataset.dataframes['alderman_votes']
votes_dataframe.head(3)

In [ ]:
# Working with multiple datasets

# Load two datasets from data.world that you'd like to merge:
int_dataset = dw.load_dataset('https://data.world/jonloyens/intermediate-data-world')
fipsCodes_dataset = dw.load_dataset('https://data.world/uscensusbureau/fips-state-codes')

# Create two dataframes
police_shootings = int_dataset.dataframes['fatal_police_shootings_data']
state_abbrvs = fipsCodes_dataset.dataframes['statesfipscodes']

## Merge the two datasets together on the state and stusab fields:
merged_dataframe = police_shootings.merge(state_abbrvs, how = 'left', left_on = 'state', right_on='stusab')

## Create a 'citystate' column in the merged_dataframe dataframe with the format `city, state_name`:
merged_dataframe["citystate"] = merged_dataframe["city"] + ", " + merged_dataframe["state_name"]

## Print head of merged dataframe
pp.pprint(merged_dataframe.head(5))

In [ ]:
# Querying data.world via SDK
## Query using SQL or SPARQL query languages. SQL is default, or add `query_type='sparql'` as a parameter for SPARQL. 
## The `query()` method gives you access to three properties to access the resulting data: `raw_data`, `table`, and `dataframe`.

# SQL

## Single table query exercise

# Define query string
sql_query = "SELECT * FROM `unhcr_all` WHERE Year = 2010"

# Query table, passing query string as parameter
query2010 = dw.query('https://data.world/nrippner/refugee-host-nations', sql_query)

# Create dataframe using dataframe property
unhcr2010 = query2010.dataframe

# Print first 5 rows of results
unhcr2010.head(5)

In [ ]:
# SQL

## Multi-table join exercise

# Define query string. Note that secondary table and fields are explicitly referenced using dataset key (ownerid/tableid)
sql_query = "SELECT state, count(fmid) as count, Avg(obesity.Value) as obesityAvg FROM Export LEFT JOIN health.`obesity-by-state-2014`.`adult_obese` as obesity ON state = obesity.location GROUP BY state ORDER BY count desc"

# Query 'local' table, passing query string as parameter
queryResults = dw.query('https://data.world/agriculture/national-farmers-markets', sql_query)

# Create dataframe using dataframe property
stateStats = queryResults.dataframe

# Plot results on state
stateStats.plot(x='state')

In [ ]:
# SPARQL
## Learn SPARQL using our tutorial at https://docs.data.world/documentation/api/sparql.html

# Define query string
sparql_query = "PREFIX GOT: <https://tutorial.linked.data.world/d/sparqltutorial/> SELECT ?FName ?LName WHERE {?person GOT:col-got-house \"Stark\" . ?person GOT:col-got-fname ?FName . ?person GOT:col-got-lname ?LName .}"

# Query table, passing query string and `query_type` as parameters
queryResults = dw.query('http://data.world/tutorial/sparqltutorial', sparql_query, query_type='sparql')

# Create dataframe using dataframe property
houseStark = queryResults.dataframe

# Print first 5 rows of results
pp.pprint(houseStark)


## Advanced SDK Functionality

The data.world Python SDK includes a variety of API wrappers, available via the `ApiClient` class, that let you create, replace, update, and delete a dataset. In this section, we’ll walk through a few common tasks:

- Use `api_client()` to get an instance of the `ApiClient`
- Create a dataset
- Add a file from a dataframe: we’ll write to a local csv and the upload the file
- Add a file from a source URL: this is an easy way to add external data to your dataset and keep it up to date. We’ll use a file from GitHub as an example, but you can use any URL source that points to a file.
- Sync the dataset: this simple call reloads any files with a source URL, to ensure the latest version.
- Update the dataset: after creating a dataset, use `update_dataset` to change attirbutes like description, summary or tags.

Use `help(api_client)` to learn more about each available function or see the full [data.world API documentation](https://docs.data.world/documentation/api/).


In [ ]:
# Create an instance of the ApiClient using `api_client()`
api_client = dw.api_client()

# See api_client documentation
help(api_client)

In [ ]:
# Create a dataset using create_dataset method. 

# Replace the < > items with your username and desired dataset title. Visibility can be changed to 'OPEN' if you choose.
api_client.create_dataset(owner_id="<YOUR_USERNAME>", title="<DATASET_TITLE>", visibility='PRIVATE')

In [ ]:
# Write a dataframe to a local file and upload to dataset

# Create dataframe
police_shootings = dw.load_dataset('https://data.world/jonloyens/intermediate-data-world').dataframes['fatal_police_shootings_data']

# Write dataframe to local csv using pandas to_csv() method
police_shootings.to_csv('police_shootings.csv', encoding='utf-8')

# Add file to your dataset using upload_files(). Replace the < > items with your dataset values
api_client.upload_files('<YOUR_USERNAME>/<DATASET_TITLE>',['police_shootings.csv'])


In [ ]:
# Update dataset

# Add a file from an external source URL. In this example we'll use github. 
# Replace the < > items with your dataset values
api_client.add_files_via_url('<YOUR_USERNAME>/<DATASET_TITLE>',{'shootings_of_police.csv': 'https://github.com/fivethirtyeight/data/blob/master/police-deaths/all_data.csv’})

# For files added with add_files_via_url, fetch the latest version using the sync() method:
api_client.sync_files('<YOUR_USERNAME>/<DATASET_TITLE>')
                                                                
# Use the update_dataset() method to update the metadata after dataset creation:
api_client.update_dataset('<YOUR_USERNAME>/<DATASET_TITLE>', description='Dataset created to test out the python SDK functionality.', tags=['test', 'datacamp'])
